In [40]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

from unityagents import UnityEnvironment

from inspect import getsource
import torch
import random
from collections import deque

import colorsys

from drl_nav.agent import Agent_PER
from drl_nav.network.heads import LabelizerNet, AuxNet, QNet
from drl_nav.network.bodies import ConvBody

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


__________

In [6]:
banana_path= r"environment/VisualBanana_Windows_x86_64/Banana.exe"
visual_banana_path= r"environment/VisualBanana_Windows_x86_64/Banana.exe"
env = UnityEnvironment(file_name=visual_banana_path)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 0
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [7]:
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
n_A = brain.vector_action_space_size
state = env_info.vector_observations[0]
n_S = len(state)

print('Number of agents:', len(env_info.agents))
print('Number of actions:', n_A)
print("State's shape:", state.shape)

Number of agents: 1
Number of actions: 4
State's shape: (0,)


______

##### Debug

In [ ]:
# 2 4 3 => 3 4 2 

In [114]:
a = np.array([
    [
        [111, 112, 113, 114],
        [121, 122, 123, 124],
        [131, 132, 133, 134],
    ],
    [
        [211, 212, 213, 214],
        [221, 222, 223, 224],
        [231, 232, 233, 234]
    ]
])
print(a.shape)

(2, 3, 4)


_____
#### Helper functions


In [8]:
def select_action(last_action, epsilon=0.75):
    """
    Exploratory action selection strategy that favor forward motion and action conservation.
    Actions explanation:
    0: forward
    1: backward
    2: left
    3: right
    """
    if np.random.random() > epsilon:
        action = np.random.choice([0, 1, 2, 3], p=[.5, .1, .2, .2])
    else:
        action = last_action
        
    return action

_____
#### One espisode of interaction

###### TODO
- make it learn
- have and optimzer, zero_grad & all
- see https://discuss.pytorch.org/t/multi-label-classification-in-pytorch/905/44

In [44]:
# labelizer = LabelizerNet()

conv_body = ConvBody()
q_net = QNet(conv_body, n_A)
aux_net = AuxNet(conv_body, n_directions=3, n_colors=2)


env_info = env.reset(train_mode=True)[brain_name]
state = env_info.visual_observations[0]    

In [45]:
%%time

env_info = env.reset(train_mode=True)[brain_name]
state = env_info.visual_observations[0]    

score, step, last_action = 0, 0 ,0

while True:
    
    action = select_action(last_action)
    last_action = action
    
    env_info = env.step(action)[brain_name]      
    next_state = env_info.visual_observations[0] 
    reward = env_info.rewards[0]                 
    done = env_info.local_done[0]                
    
    score += reward                               
    state = next_state  
    step += 1
    
    # step
    state_torch = torch.from_numpy(np.moveaxis(state, 3, 1)).float()
    
    yellow_labels, blue_labels = labelizer(torch.squeeze(state_torch))
    colors_labels = np.concatenate((yellow_labels, blue_labels ))
    color_pred = aux_net(state_torch)
    
    
    if (done | step > 100):                                      
        break
    
print("Score: {}".format(score))

Score: 0.0
Wall time: 4.56 s
